# [3.3] Running evals with the inspect library

# 1️⃣ Introduction to Inspect

## Learning Objectives

- Understand how **solvers** and **scorers** in the inspect framework function
- Familiarise yourself with **plans**, which ultimately are just a list of solver functions, and then a scorer function.
- Understand how inspect uses **record_to_sample** to modify json datasets into evaluation datasets.
- Know the basics of how all of these classes come together to form a **Task** object.

In [39]:
import os
import json
from typing import Literal
from inspect_ai import Task, eval, task
from inspect_ai.log import read_eval_log
from utils import omit
from inspect_ai.dataset import FieldSpec, json_dataset, Sample, example_dataset
from inspect_ai.solver._multiple_choice import Solver, solver, Choices, TaskState, answer_options
from inspect_ai.solver._critique import DEFAULT_CRITIQUE_TEMPLATE, DEFAULT_CRITIQUE_COMPLETION_TEMPLATE
from inspect_ai.scorer import (model_graded_fact, match, answer, scorer)
from inspect_ai.scorer._metrics import (accuracy)
from inspect_ai.solver import ( chain_of_thought, generate, self_critique,system_message, Generate )
from inspect_ai.model import ChatMessageUser, ChatMessageSystem, ChatMessageAssistant, get_model
import random
import jaxtyping 
from itertools import product

## Inspect
Inspect (or inspect_ai) is a library written by the UK AISI in order to streamline model evaluations. Inspect provides the following in order to make writing evals easier:

- It automates the creation of logging files, and stores a lot of useful information about the evals that we run for us.

- It provides nice "scoring" functions (and some of its automated solver functions also come in handy, but mostly we will write our own).

- It provides a nice layout to view the results of our evals, so we don't have to look directly at model outputs (which can be very gross, as we will find out when we work with agents).


### An Overview of Inspect


The main object of inspect is a `Task` object, which will contain all of the information about the eval we'll run on the model, including:

- The dataset we will evaluate the model on. This is essentially a list of `Sample` objects, which is how the inspect library stores individual questions. We will write a function to turn our list of questions formatted as a json file into a list of questions formatted as `Sample` objects.

- The `plan` that the evaluation will proceed along. This is a list of functions which inspect calls `Solver` functions. `Solver` functions can modify the evaluation questions; get the model to generate a response; or do both at the same time. A typical collection of solvers to form `plan` list might look like:

    - A `chain_of_thought` function which will add an indication that the model should use chain of thought before answering.

    - A `generate` function that calls the LLM API to generate a response to the question (now also with the chain of thought prompt).

    - A `self_critique` function that takes all of the output so far, and attaches a prompt asking the model to critique its own output.

    - Another `generate` call which calls the LLM API to generate an output in response to the `self_critique` prompt.

- The final component of a `Task` object is a `scorer` function, which tells the inspect library how it should evaluate the model's output.

![image file](Inspect%20Big%20Picture.png)

### Dataset

A dataset is built out of `Sample` objects. We'll have one `Sample` object for each question in our evaluation. They contain basically the exact same information as we've constructed for each question in our list, but renamed and reformatted so that they fit in with the way the rest of the inspect library is built. This means that turning our list of questions in json format into a `Dataset` of `Sample` objects just requires us to handle the field-mapping. The components of the `Sample` object are:

- The `input` to the model. This is a `ChatMessage` list of dictionaries which contains the content which we will feed to the model during the eval consisting of:
    - A system message

    - A user message (this is the most necessary part of all)

- A `target` which represents the "correct" answer (or `answer_matching_behavior` in our context).

<details> 
<summary>Aside: Longer ChatMessage lists</summary>
<br>
For more complicated evals, we're able to do an arbitrary length list of ChatMessages including: 
<ul> 
<li>Multiple user messages and system messages.</li>
&nbsp;
<li>"Assistant messages" that the model will believe it has produced in response to user and system messages. However we can write these ourselves to trick the model.</li>
&nbsp;
<li>"Tool call messages" which can mimic the model's interaction with any tools that we give it. We'll learn more about tools in the agent evals section.</li>
</ul>
</details>

Since we are constructing a multiple choice evaluation, we will also need to feed it `choices`. Then it also takes in `metadata`, which can consist of any data we might want to store related to the question in our logs, for example:

- If there are multiple categories of question, and we want to be able to plot results by category.

- If we are going to conduct the evaluation with/without a system prompt

- Any other information about questions that we want to be easily accessible from the log files.

### Exercise - Write record_to_sample function

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 minutes on this exercise.
```

This function will take in a single element of your list of json objects, and returns a Sample object.

In [5]:
def record_to_sample(record : dict) -> Sample:
    return Sample(
        input = [
            {
                'role' : "system",
                "content" : record["system"]
            },
            {
                'role' : "user",
                "content" : record["question"]
            }
        ],
        target = record["answer_matching_behavior"],
        choices =  list(record["answers"].values()),
        metadata = {"label" : record["label"], "system_prompt" : True}
    )

Now we use the `json_dataset()` function to apply this function to the list of questions in our dataset.

In [10]:
eval_dataset = json_dataset("C:\\Users\\styme\\OneDrive\\Documents\\AI STUFF\\Model Written Evals\\Code Replication\\ARENA_evals\\day1-3_dataset_generation\\datasets\\2c-generated-final.json", record_to_sample)

### Solvers

Solvers are functions which outline how the eval will be presented to the model. They can be used for:

- Providing a system prompt.
 
- Providing the evaluation question.

- Prompt engineering (e.g. chain of thought, asking the model to output in certain formats, telling the model how to act when answering).

- Self-critique (where the model criticises its prior answer to see if it arrives at a different conclusion).

- Providing fake assistant messages to the model (useful if you want to conduct an eval in the midst of an ongoing conversation with a model).

Some built-in solvers which accomplish these are:

- `generate()` which is the most fundamental solver function. This just calls the LLM API to generate a response to the evaluation question.

- `system_message()` which adds a system message to the list of ChatMessages.

- `prompt_template()` which modifies the user prompt by substituting the current prompt into the `{prompt}` placeholder within the specified template. Can also substitute variables defined in the `metadata` of any Sample object.

- `chain_of_thought()` which modifies the user prompt by substituting the current prompt into the `{prompt}` placeholder in order to add a standard chain_of_thought template. By default, this template is:

    ```python
    DEFAULT_COT_TEMPLATE = r"""
    {prompt}

    Before answering, reason in a step-by-step manner as to get the right answer. Provide your answer at the end on its own line in the form "ANSWER: $ANSWER" (without quotes) where $ANSWER is the answer to the question.
    """
    ```

- `multiple_choice()` which presents a multiple choice question (with a single correct answer) in the format:
    ```python
    SINGLE_ANSWER_TEMPLATE = r"""
    Answer the following multiple choice question. The entire content of your response should be of the following format: 'ANSWER: $LETTER' (without quotes) where LETTER is one of {letters}.

    {question}

    {choices}
    """.strip()
    ```
    where `{choices}` are formatted as ```"A) Choice 1\nB) Choice 2\nC) Choice 3"```. This solver then automatically calls generate to call the LLM API for a response to the multiple choice question.

We'll design our own solvers in the next section.

## Scorers

Scorers are what the inspect library uses to evaluate model output. We use them to determine whether the model's answer is the `target` answer. From the [inspect documentation](https://inspect.ai-safety-institute.org.uk/scorers.html):


><br>
>Scorers generally take one of the following forms:
>
>1. Extracting a specific answer out of a model’s completion output using a variety of heuristics.
>
>2. Applying a text similarity algorithm to see if the model’s completion is close to what is set out in the `target`.
>
>3. Using another model to assess whether the model’s completion satisfies a description of the ideal answer in `target`.
>
>4. Using another rubric entirely (e.g. did the model produce a valid version of a file format, etc.)
>
><br>

Since we are writing a multiple choice benchmark, the scorer we will generally use is called `match()`. This determines whether the `target` appears at the end of the model's output (or the start, but by default it will look at the end). The code for this scorer function can be found [in the inspect_ai github](https://github.com/UKGovernmentBEIS/inspect_ai/blob/c59ce86b9785495338990d84897ccb69d46610ac/src/inspect_ai/scorer/_match.py). 

<details><summary>Aside: some other scorer functions:</summary>
<br>
Some other scorer functions that would be useful in different evals are:

- `includes()` which determines whether the target appears anywhere in the model output.

- `pattern()` which extracts the answer from the model output using a regular expression.

- `answer()` which is a scorer for model output when the output is preceded with "ANSWER:".

- `model_graded_qa()` which has another model assess whether the output is a correct answer based on guidance in `target`. This is especially useful for evals where the answer is more open-ended. Depending on what sort of questions you're asking here, you may want to change the prompt template that is fed to the grader model.
<br>
</details>

### The Task Object

The dataset, solvers, and scorers come together in a Task object. This contains everything we need to run the evaluation, and we will be able to call Inspect's `eval` function, choose a model to evaluate, and Inspect will take care of the rest. For example, a Task object might look like:

In [11]:
from inspect_ai.dataset import example_dataset

@task
def theory_of_mind():
    return Task(
        dataset=example_dataset("theory_of_mind"),
        plan=[
          chain_of_thought(), 
          generate(), 
          self_critique(model = "openai/gpt-3.5-turbo")
        ],
        scorer=model_graded_fact(model = "openai/gpt-3.5-turbo"),
    )


Now let's see what it looks like to run this example eval through inspect. We'll only run it for 10 samples on gpt-3.5-turbo to minimise the number of OpenAI credits we use.

In [12]:
log = eval(theory_of_mind(),
              model = "openai/gpt-3.5-turbo",
              limit=10,
              log_dir="./gitignore/logs" # TODO: Figure out gitignore stuff from Sunischal's advice
    )

Output()

Finally, we can view the results of the log in Inspect's log viewer. To do this, run the code below. It will locally host a display of the results of the example evaluation at http://localhost:7575. You need to modify "`your_log_name`" in the --log-dir argument below so that it matches the log name that was output above (which is dependent on the date and time).

<details><summary>Aside: Log names</summary> I'm fairly confident (but not 100%) that when Inspect accesses logs, it utilises the name, date, and time information as a part of the way of accessing and presenting the logging data. Therefore it seems that there is no easy way to rename log files to make them easier to access (I tried it, and Inspect didn't let me open them). If someone works out how to change the names of logs then let me know. </details>

In [16]:
!inspect view --log-dir "./gitignore/logs/2024-08-19T15-20-15+01-00_theory-of-mind_UCqYeegADxYZBDbG8FLZhJ.json" --port 7575

^C


Inspect view running at http://localhost:7575/


## 2️⃣ Writing Solvers 

Despite the utility of Inspect, it is lacking some solvers which we'll want to use, and some solvers which it gives to us are utilised in ways that won't really align with the way that we're running our evaluations. For this reason, we'll write our own solvers.

Solvers modify the `TaskState`. This is how Inspect stores the state of each question. There is a different `TaskState` class initialised for every new question during our eval. It contains things like the initial question (stored as a string under state.user_prompt.text), and the ChatMessageHistory (stored as a list under state.messages).

Solvers have the potential to take quite a long time to run. For example, the `self_critique` solver needs to wait for the model to generate a response before it can modify the `TaskState` with the criticism. For this reason, Inspect calls solvers using the `await` key word. So when we define a solver, we need to define the modification of the TaskState as its own asynchronous `solve` function.


### Exercise - Write a simple solver function
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-15 minutes on this exercise.
```

Write a solver function that checks if `state` has a `user_prompt`, and modifies the `user_prompt` according to a template string which contains `"{question}"` — where the original `user_prompt` will get placed in the template.

You might find Inspect's [solver documentation](https://inspect.ai-safety-institute.org.uk/solvers.html), or even their [github source code](https://github.com/UKGovernmentBEIS/inspect_ai/tree/0d9f9c8d5b472ec4f247763fe301a5be5839a607/src/inspect_ai/solver) useful for this exercise (and the following exercise).

In [29]:

def prompt_template(template : str, **params : dict) -> Solver:
    async def solve(state : TaskState, generate: Generate) -> TaskState:
        if state.user_prompt:
            state.user_prompt.text = template.format(
                prompt=state.user_prompt.text, **params
            )
        return state

    return solve

### Exercise - Write other solver functions
```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 20-25 mins on this exercise.
```

Now write a greater variety of solver functions that we can use in our evaluations of the model.

In [34]:
ChatMessageTemplate = multiple_choice_2_template = {"role" : "user",
                              "content" : "Now provide your conclusion in the form ANSWER: $LETTER where $LETTER is one of A, B. Only output in this format."}
print(ChatMessageSystem(content = "hello"))

content='hello' source=None role='system' tool=None


In [ ]:
@solver
def multiple_choice_format(template : str, **params : dict) -> Solver:
    '''
    This modifies the initial prompt to be in the format of a multiple choice question. Make sure that {question} and {choices} are in the template string, so that you can format those parts of the prompt.
    '''
    assert r"{choices}" in template and r"{question}" in template,r"ERROR: The template must contain {question} or {choices}."

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        if state.user_prompt:
            state.user_prompt.text = template.format(
                question=state.user_prompt.text,
                choices=answer_options(state.choices),
                **params,
            )
        return state
    
    return solve

@solver
def choose_multiple_choice(ChatMessageTemplate : dict, **params : dict) -> Solver:
    '''
    This modifies the ChatMessageHistory to add a message at the end indicating that the model should make a choice out of the options provided. 
    
    Make sure the model outputs so that its response can be extracted by the "match" solver. You will have to use Inspect's ChatMessageUser class for this.
    '''
    async def solve(state: TaskState, generate: Generate) -> TaskState:
        if state.messages:
            state.messages.append(ChatMessageUser(**ChatMessageTemplate))
        return state
    return solve

@solver
def system_message(system_message : str, **params : dict) -> Solver:
    '''
    This solver is used to provide a fixed system prompt to the model.

    This is built into Inspect, so if you're happy to use their version without any mofifications, you can just use Inspect's system_message solver.
    '''
    def insert_system_message(messages : list, message: ChatMessageSystem) -> None:
        lastSystemMessageIndex = -1
        for i in list(reversed(range(0,len(messages)))):
            if isinstance(messages[i],ChatMessageSystem):
                lastSystemMessageIndex = i
                break
        messages.insert(lastSystemMessageIndex+1,message)
    async def solve(state : TaskState, generate : Generate) -> TaskState:
        insert_system_message(state.messages, ChatMessageSystem(content = system_message))
        return state
    return solve

@solver
def self_critique(model : str, critique_template : str, critique_completion_template : str, **params) -> Solver:
    '''
    This solver is used get the model to evaluate its own response to the task. Then appends its criticism as a user message. 
    
    This is built into Inspect, so if you're happy to make use their version without any modifications, you can just use Inspect's self_critique solver.
    '''
    model = get_model(model)
    async def solve(state : TaskState, generate: Generate) -> TaskState:
        metadata = omit(state.metadata, ["question", "completion", "critique"])
        critique = await model.generate(
            critique_template.format(
                question=state.input_text,
                completion=state.output.completion,
                **metadata,
            )
        )
        
        state.messages.append(
            ChatMessageUser(
                content = completion_template.format(
                    question = state.input_text,
                    completion = state.output.completion,
                    critique = critique.completion,
                    **metadata,
                ),
            )
        )

        return await generate(state)
    return solve

# Now you should be able to write any other solvers that might be useful for your specific evaluation task (if you can't already use any of the existing solvers to accomplish the evaluation you want to conduct). For example:
    #
    # A "language" template, which tells the model to respond in diffeent languages (you could even get the model to provide the necessary translation first).
    #
    # 
    #
    # A solver which provides "fake" reasoning by the model where it has already reacted in certain ways, to see if this conditions the model response in different ways. You would have to use the ChatMessageUser and ChatMessageAssistant Inspect classes for this
    #
    #
    #   
    #  maybe.
    #
#  Do so here!






## 3️⃣ Writing Tasks and evaluating

- Benchmarking. May need to write different solvers/definitely need to write different dataset generation procedures to handle system prompts.
- Writing then running the overall benchmarking task
- Writing then running the overall evaluation task procedure.


Before we can finalize our evaluation task, we'll need to modify our dataset generation from earlier. The issue with the function we wrote is twofold:

- First of all, we didn't shuffle the answers. Your dataset may have been generated pre-shuffled, but if not, then you ought to be aware that the model has a bias towards the first answer. See for example [Narayanan & Kapoor: Does ChatGPT have a Liberal Bias?](https://www.aisnakeoil.com/p/does-chatgpt-have-a-liberal-bias) To mitigate this confounder, we should shuffle the answers around when loading in the dataset.

<details><summary>Aside: Inspect's shuffle function</summary> 

Inspect comes with a shuffle function for multiple choice questions. However, they then automatically "unshuffle" the responses back to how they are presented in your dataset. 

I'm sure there are use cases for this, but in our case its often unnecessarily confusing to see the model reasoning about e.g. how much it thinks option B is the correct answer, and then Inspect telling you the model selected "option A" since its selection choice gets unshuffled but its reasoning doesn't.

For this reason, and because it's not very hard to do, we will just build our own shuffler that doesn't do any unshuffling.</details>

- Secondly, this function can only load questions with the corresponding system prompt. It is often useful to load the dataset without a system prompt (for example, this can be useful for benchmarking, or even when running an evaluation, we may want to see how the model reacts to our evaluation with a more generic "You are a harmless, helpful, honest AI Chat Assistant"). 

### Exercise - Write record_to_sample function with shuffling
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪
```

Take the record_to_sample function you wrote in section 1, and modify it so that it randomly shuffles the order in which answers are presented. Remember to correspondingly shuffle the `target`.

In [ ]:
def record_to_sample_shuffle(record : dict, shuffle : bool) -> Sample:
    shuffle = random.choice([True,False])
    return Sample(
        input = [
                    {
                        "role" : "system",
                        "content" : record["system"] if system_prompt == True else "You are a harmless, helpful, and honest AI chat assistant."
                    },
                    {
                        "role" : "user", 
                        "content" : record["question"] 
                    }
                ],
        target = record["answer_not_matching_behavior"][0] if shuffle else record["answer_matching_behavior"][0],
        choices = list(reversed(record["answers"].values())) if shuffle else list(record["answers"].values()),
        metadata = {"label" : record["label"], "system_prompt" : True}
    )

### Exercise - Write record_to_sample function to handle system prompts differently
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵⚪⚪⚪⚪

```

There are two reasons we might not want the system prompt provided to the model in the usual way. Either:
- We might want the model not to have the context of the system prompt, and give it a different (fixed) context entirely (which we can do via the system_prompt solver we wrote in the section 3).

- During benchmarking (NEED TO EXPLAIN BENCHMARKING SOMEWHERE HERE PROBABLY), we might not want the model to provide an answer with the system prompt as its actual system prompt, but we still want to provide this system prompt context somewhere in the user prompt.

Write a version of the `record_to_sample` function which handles system prompts in each of these two ways:

In [ ]:
def record_to_sample_no_system_prompt(record : dict, shuffle : bool) -> Sample:
    shuffle = random.choice([True,False])
    return Sample(
        input = [
            {
                'role' : "user",
                "content" : record["question"]
            }
        ],
        target = record["answer_not_matching_behavior"][0] if shuffle else record["answer_matching_behavior"][0],
        choices = list(reversed(record["answers"].values())) if shuffle else list(record["answers"].values()),
        metadata = {"label" : record["label"], "system_prompt" : False}
    )

def record_to_sample_system_prompt_as_context(record : dict, shuffle : bool) -> Sample:
    shuffle = random.choice([True,False])
    return Sample(
        input = [
            {
                'role' : "user",
                "content" : "Context:" + record["system"] + "\n\n" + record["question"]
            },
        ],
        target = record["answer_not_matching_behavior"][0] if shuffle else record["answer_matching_behavior"][0],
        choices = list(reversed(record["answers"].values())) if shuffle else list(record["answers"].values()),
        metadata = {"label" : record["label"], "system_prompt" : False}
    )

Since `json_dataset` can only take a function which takes one argument `record`, we need to wrap any `record_to_sample` functions which take optional arguments (like `shuffle`) so that the wrapper takes just one function. This is done in the code below.

In [ ]:
def record_to_sample(system, system_prompt_as_context, shuffle):
    assert not system and not system_prompt_as_context, "ERROR: You can only set one of system or system_prompt_as_context to be True."
    def wrapper(record):
        if system:
            return record_to_sample(record, shuffle)
        elif system_prompt_as_context:
            return record_to_sample_system_prompt_as_context(record, shuffle)
        else:
            return record_to_sample_no_system_prompt(record, shuffle)
    return wrapper

Now we can load our dataset in using any one of these functions.

In [ ]:
eval_dataset_system = json_dataset("your/path/to/eval_dataset.json", record_to_sample(system = True, system_prompt_as_context = False, shuffle = True))
eval_dataset_no_system = json_dataset("your/path/to/eval_dataset.json", record_to_sample(system = False, system_prompt_as_context = False, shuffle = True))
eval_dataset_system_as_context = json_dataset("your/path/to/eval_dataset.json", record_to_sample(system = False, system_prompt_as_context = True, shuffle = True))

### Exercise - Writing Prompts and Building out your eval task
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵⚪⚪⚪⚪

You should spend
```

Now we want to build out the tasks we'll ultimately test the model on.

A few things:

- Here you really need to sort out prompts
- You need to figure out how to benchmarking
    - You need to figure out how you're going to deal with Claude being a wimp (well done Anthropic, I guess).
- You need to hammer out what evals you want to conduct, and in what way.

## 4️⃣ Dealing with logs and plotting